# Task 3: Airflow MongoDB + SparkSQL

DDS Group Project members: Andrea Quiroz, Chelyah Miller, Amit Chaubey, Helen Lin, Swati Agarwal

In [1]:
import pyspark
from pyspark.sql import Row, SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window

In [2]:
sc = SparkContext.getOrCreate()

25/03/07 16:14:28 WARN Utils: Your hostname, Andreas-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 10.0.0.160 instead (on interface en0)
25/03/07 16:14:28 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/07 16:14:28 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
ss = SparkSession.builder.getOrCreate()

## Create Dataframes from the data from MongoDB Atlas

### Loading in the data from the json file that we grabbed from our MongoDB Atlas

In [4]:
path = "/Users/andreamellany/Documents/Documents - Andrea’s MacBook Pro/MSDS/msds697/movies_with_ratings.json"

In [6]:
movies_ratingsdf = ss.read.option("multiline", "true").json(path)

Printing the schema out:

In [7]:
movies_ratingsdf.printSchema()

root
 |-- _id: struct (nullable = true)
 |    |-- $oid: string (nullable = true)
 |-- overview: string (nullable = true)
 |-- popularity: double (nullable = true)
 |-- ratings: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- rated_at: string (nullable = true)
 |    |    |-- rating: long (nullable = true)
 |    |    |-- user: string (nullable = true)
 |-- release_date: string (nullable = true)
 |-- title: string (nullable = true)
 |-- tmdb_id: long (nullable = true)
 |-- vote_average: double (nullable = true)
 |-- vote_count: long (nullable = true)



Checking out the dataframe:

In [8]:
movies_ratingsdf.show()

+--------------------+--------------------+----------+--------------------+------------+--------------------+-------+------------+----------+
|                 _id|            overview|popularity|             ratings|release_date|               title|tmdb_id|vote_average|vote_count|
+--------------------+--------------------+----------+--------------------+------------+--------------------+-------+------------+----------+
|{67b7d9d43806d131...|Sonic, Knuckles, ...|  3084.368|[{2025-01-26T09:2...|  2024-12-19|Sonic the Hedgehog 3| 939243|       7.783|      1555|
|{67b7d9d43806d131...|Kraven Kravinoff'...|   1785.56|[{2024-12-22T03:3...|  2024-12-11|   Kraven the Hunter| 539972|         6.6|      1066|
|{67b7d9d43806d131...|After receiving a...|  1427.587|[{2025-01-29T17:1...|  2024-11-21|             Moana 2|1241982|       7.208|      1311|
|{67b7d9d43806d131...|Fifteen years aft...|  1312.816|[{2025-01-21T20:1...|  2025-01-15|      Back in Action| 993710|       6.633|       793|
|{67b7

## Run SparkSQL SQL queries over the data frames

In [10]:
# Creating the dataframe in order to make temporary SQL data to run SparkSQL queries:
movies_ratingsdf.createOrReplaceTempView("mr_table")

### Query 1:
Average rating, total rating count, and standard deviation of ratings per movie

In [23]:
query1 = ss.sql("SELECT title, \
    COALESCE(AVG(r.rating), 0)  AS avg_rating, \
    COUNT(r.rating) AS total_ratings, \
    STDDEV(r.rating) AS rating_stddev \
    FROM mr_table \
    LATERAL VIEW OUTER explode(ratings) exploded_ratings AS r \
    WHERE size(ratings) > 0 \
    GROUP BY title \
    ORDER BY avg_rating DESC")
query1.show()

+--------------------+-----------------+-------------+------------------+
|               title|       avg_rating|total_ratings|     rating_stddev|
+--------------------+-----------------+-------------+------------------+
|         Sex Stories|             10.0|            1|              NULL|
|      The Prosecutor|             10.0|            1|              NULL|
|No Manches Frida ...|             10.0|            1|              NULL|
|Prison Break: The...|             10.0|            1|              NULL|
|              Murari|             10.0|            1|              NULL|
|      The Red Thread|             10.0|            1|              NULL|
|My Little Pony: A...|             10.0|            2|               0.0|
| Murmur of the Heart|             10.0|            1|              NULL|
|One Piece: Episod...|             10.0|            1|              NULL|
| Detective Chinatown|             10.0|            1|              NULL|
|           Dominique|             10.

### Query 2:
Number of distinct users who rated each movie that has ratings! Excluded movies with 0 ratings

In [24]:
query2 = ss.sql("SELECT title, \
    COUNT(DISTINCT r.user) AS distinct_user_count \
    FROM mr_table \
    LATERAL VIEW OUTER explode(ratings) exploded_ratings AS r \
    GROUP BY title \
    ORDER BY distinct_user_count DESC")
query2.show()

+--------------------+-------------------+
|               title|distinct_user_count|
+--------------------+-------------------+
|            Deadpool|                890|
|Guardians of the ...|                876|
|           Inception|                855|
|        Interstellar|                854|
|     The Dark Knight|                841|
|        The Avengers|                805|
|          The Matrix|                771|
|              Frozen|                707|
|         The Martian|                695|
|          Fight Club|                687|
|              Avatar|                686|
|  Mad Max: Fury Road|                683|
|      Doctor Strange|                682|
|            Iron Man|                679|
|The Lord of the R...|                666|
|Captain America: ...|                660|
|Avengers: Age of ...|                657|
|The Dark Knight R...|                649|
|Captain America: ...|                645|
|Star Wars: The Fo...|                639|
+----------

### Query 3:
"Polarizing" movies, basically controversial movies. This ranks movies by the range of ratings (diff between the max rating and min rating). A higher range means that some people gave very low scores while others gave very high scores!

In [28]:
query3 = ss.sql("SELECT title, \
                (MAX(r.rating) - MIN(r.rating)) AS rating_range \
            FROM mr_table \
            LATERAL VIEW OUTER explode_outer(ratings) exploded_ratings AS r \
            GROUP BY title \
            ORDER BY rating_range DESC") 
query3.show()

+--------------------+------------+
|               title|rating_range|
+--------------------+------------+
|The Final Destina...|           9|
|           The Whale|           9|
|        The Proposal|           9|
|          The Ritual|           9|
|Salaar: Part 1 - ...|           9|
|  The Last Airbender|           9|
|           Sex Drive|           9|
|Before I Go to Sleep|           9|
|            Warcraft|           9|
|          The Nun II|           9|
|   Failure to Launch|           9|
|       A Quiet Place|           9|
|The Bridges of Ma...|           9|
|    Now You See Me 2|           9|
|   Kill Bill: Vol. 2|           9|
|Birdman or (The U...|           9|
|American Pie Pres...|           9|
|        Daddy's Home|           9|
|   Bohemian Rhapsody|           9|
|         The Tourist|           9|
+--------------------+------------+
only showing top 20 rows



### Query 4:
Rating distribution for every movie. 

In [29]:
query4 = ss.sql("SELECT title, \
                    r.rating, \
                    COUNT(*) AS rating_count \
                FROM mr_table \
                LATERAL VIEW explode(ratings) exploded_ratings AS r \
                GROUP BY title, r.rating \
                ORDER BY title, r.rating") 
query4.show()

+--------------------+------+------------+
|               title|rating|rating_count|
+--------------------+------+------------+
|              #Alive|     3|           1|
|              #Alive|     4|           2|
|              #Alive|     6|          13|
|              #Alive|     7|          12|
|              #Alive|     8|           2|
|              #Alive|     9|           1|
|              #Alive|    10|           2|
|         #Blue_Whale|     4|           1|
|         #Blue_Whale|     6|           1|
|(500) Days of Summer|     1|           4|
|(500) Days of Summer|     2|           1|
|(500) Days of Summer|     3|           2|
|(500) Days of Summer|     4|           1|
|(500) Days of Summer|     5|          13|
|(500) Days of Summer|     6|          26|
|(500) Days of Summer|     7|          48|
|(500) Days of Summer|     8|          84|
|(500) Days of Summer|     9|          29|
|(500) Days of Summer|    10|          33|
| 10 Cloverfield Lane|     1|           2|
+----------

### Query 5:
Movies that never received ratings below a certain threshold. Because our rating scale is from 1-10, I will make the threshold 6, midpoint threshold is too low in my opinion, I need to raise the standards up just a tiny bit. 

In [31]:
query5 = ss.sql("SELECT title \
                FROM mr_table \
                LATERAL VIEW explode(ratings) exploded_ratings AS r \
                GROUP BY title \
                HAVING MIN(r.rating) >= 6") 
query5.show()

+--------------------+
|               title|
+--------------------+
|         Deep Rising|
|The Rifleman of t...|
|          Undercover|
|Tom and Jerry: Th...|
|         The Verdict|
|   Memoir of a Snail|
|     Death Whisperer|
|Rurouni Kenshin: ...|
|           Capernaum|
|       La Dolce Vita|
|                Hope|
|I Want to Eat You...|
|                Toma|
|The Count of Mont...|
|   Naked Ambition 3D|
|      The Wild Bunch|
|    New Police Story|
|      The Prosecutor|
|The Trapped 13: H...|
|            Fidelity|
+--------------------+
only showing top 20 rows



## Build machine learning algorithms (optional) to derive analytics on top of the original and aggregated datasets

Due to the pace of the module, we are slowing down our timeline towards our end goal: A movie recommendation system. Andrea, Amit, Chelyah, and Helen have agreed to move forward with the project until we have reach L4. We are all very enthusiastic to see the end goal, with priority of automating our pipeline, and having at the end of it a user interface for people to interact with our project. Because some of us are continuing on with the DE concentration, we remain accountable to ourselves and Mahesh to see this come to fruition!